## Generamos el grafo

In [1]:
# Subimos el archivo de contenedores_resto_Valencia_filtrado


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import osmnx as ox
from shapely.geometry import LineString, Point
import networkx as nx
import folium

contenedores_resto = "Contenedores_resto_FCC_valencia_filtrado.csv"

contenedores_resto_filtrado_df= pd.read_csv(contenedores_resto, sep=',', engine='python')
print(f"Número de contenedores de resto: {contenedores_resto_filtrado_df.count()}")

display(contenedores_resto_filtrado_df.sample(5))

Número de contenedores de resto: calleempre           1797
nº_contenedores      1797
Lateral Derecha      1797
Lateral Izquierda    1797
Trasera              1797
lat                  1797
lon                  1797
dtype: int64


,calleempre,nº_contenedores,Lateral Derecha,Lateral Izquierda,Trasera,lat,lon
966,"C/ JOSEP GROLLO, 105",2,2,0,0,39.493362,-0.392365
1238,"C/ PINTOR FERRER CALATAIUD, 13",2,2,0,0,39.467532,-0.337671
210,"AV MESTRE RODRIGO (MÚSIC), 6",1,1,0,0,39.477078,-0.397041
1241,"C/ PINTOR GENARO LAHUERTA, 23",2,2,0,0,39.483014,-0.368557
1161,"C/ PALMA DE GANDIA, 65",1,0,0,1,39.528710,-0.384273


In [ ]:
# Descargamos el grafo de calles para coches de Valencia


G = ox.graph_from_place("Valencia, Spain", network_type='drive', simplify=True)

G = ox.add_edge_speeds(G)        # km/h
G = ox.add_edge_travel_times(G)  # segundos

largest_cc = max(nx.strongly_connected_components(G), key=len)
G_sub = G.subgraph(largest_cc).copy()

# Definimos la función para insertar un contenedor en el grafo de la ciudad de Valencia
def insertar_contenedor_en_grafo(G, contenedor_id, lat, lon):
    from shapely.geometry import Point, LineString

    point = Point(lon, lat)
    u, v, key = ox.nearest_edges(G, lon, lat)
    data_uv = G.get_edge_data(u, v, key)

    # Geometría del edge original
    if 'geometry' in data_uv:
        edge_geom = data_uv['geometry']
    else:
        edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])])

    # Proyectar el punto sobre la geometría
    nearest_point = edge_geom.interpolate(edge_geom.project(point))
    x_new, y_new = nearest_point.x, nearest_point.y

    # Crear nodo contenedor
    new_node = f"contenedor_{contenedor_id}"
    G.add_node(new_node, x=x_new, y=y_new, contenedor=True)

    # Crear geometrías para las nuevas aristas
    geom_uv = LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (x_new, y_new)])
    geom_vu = LineString([(x_new, y_new), (G.nodes[v]['x'], G.nodes[v]['y'])])
    length_uv, length_vu = geom_uv.length, geom_vu.length

    # Atributos comunes
    attrs_base = data_uv.copy()
    for attr in ['geometry', 'length', 'travel_time']:
        attrs_base.pop(attr, None)

    speed = data_uv.get('speed_kph', 30)
    travel_time_uv = length_uv / (speed * 1000 / 3600)
    travel_time_vu = length_vu / (speed * 1000 / 3600)

    # Eliminar cualquier arista existente u→v o v→u
    if G.has_edge(u, v, key):
        G.remove_edge(u, v, key)
    if G.has_edge(v, u):
        for rev_key in list(G.get_edge_data(v, u).keys()):
            G.remove_edge(v, u, rev_key)

    # Añadir nuevas aristas obligando a pasar por el contenedor
    # u -> contenedor -> v
    G.add_edge(u, new_node, **dict(attrs_base, length=length_uv, travel_time=travel_time_uv, geometry=geom_uv))
    G.add_edge(new_node, v, **dict(attrs_base, length=length_vu, travel_time=travel_time_vu, geometry=geom_vu))

    # v -> contenedor -> u (simétrico)
    geom_vc = LineString([(G.nodes[v]['x'], G.nodes[v]['y']), (x_new, y_new)])
    geom_cu = LineString([(x_new, y_new), (G.nodes[u]['x'], G.nodes[u]['y'])])
    length_vc, length_cu = geom_vc.length, geom_cu.length
    travel_time_vc = length_vc / (speed * 1000 / 3600)
    travel_time_cu = length_cu / (speed * 1000 / 3600)

    G.add_edge(v, new_node, **dict(attrs_base, length=length_vc, travel_time=travel_time_vc, geometry=geom_vc))
    G.add_edge(new_node, u, **dict(attrs_base, length=length_cu, travel_time=travel_time_cu, geometry=geom_cu))

    return G

i = 0

for _, row in contenedores_resto_filtrado_df.iterrows():
    G_sub = insertar_contenedor_en_grafo(G_sub, row["calleempre"], row["lat"], row["lon"]) 
    i += 1
    print(f"Nodos interpolados: {i}")

gdf_edges = ox.graph_to_gdfs(G_sub, nodes=False, edges=True)
gdf_nodes = ox.graph_to_gdfs(G_sub, nodes=True, edges=False)

m = folium.Map(location=[39.4699, -0.3763], zoom_start=14)

for _, row in gdf_edges.iterrows():
    if row['geometry'].geom_type == 'LineString':
        coords = [(lat, lon) for lon, lat in row['geometry'].coords]
        folium.PolyLine(coords, color='black', weight=1, opacity=0.6).add_to(m)

contenedores = gdf_nodes[gdf_nodes['contenedor'] == True]
for _, row in contenedores.iterrows():
    folium.CircleMarker(
        location=[row['y'], row['x']],
        radius=6,
        color='blue',
        fill=True,
        fill_opacity=0.8,
        tooltip=f"Contenedor: {row.name}"
    ).add_to(m)

m.save("contenedores_resto_FCC_valencia_grafo.html")
print("Mapa guardado como mapa_grafo_contenedores.html")


ox.save_graphml(G_sub, filepath="contenedores_resto_FCC_valencia_filtrado_grafo.graphml")


Nodos interpolados: 1
Nodos interpolados: 2
Nodos interpolados: 3
Nodos interpolados: 4
Nodos interpolados: 5
Nodos interpolados: 6
Nodos interpolados: 7
Nodos interpolados: 8
Nodos interpolados: 9
Nodos interpolados: 10
Nodos interpolados: 11
Nodos interpolados: 12
Nodos interpolados: 13
Nodos interpolados: 14
Nodos interpolados: 15
Nodos interpolados: 16
Nodos interpolados: 17
Nodos interpolados: 18
Nodos interpolados: 19
Nodos interpolados: 20
Nodos interpolados: 21
Nodos interpolados: 22
Nodos interpolados: 23
Nodos interpolados: 24
Nodos interpolados: 25
Nodos interpolados: 26
Nodos interpolados: 27
Nodos interpolados: 28
Nodos interpolados: 29
Nodos interpolados: 30
Nodos interpolados: 31
Nodos interpolados: 32
Nodos interpolados: 33
Nodos interpolados: 34
Nodos interpolados: 35
Nodos interpolados: 36
Nodos interpolados: 37
Nodos interpolados: 38
Nodos interpolados: 39
Nodos interpolados: 40
Nodos interpolados: 41
Nodos interpolados: 42
Nodos interpolados: 43
Nodos interpolados: 